In [1]:
pip install trimesh

     |████████████████████████████████| 639 kB 5.2 MB/s 


In [2]:
import os
import random
import glob
import trimesh
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import scipy 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv3D, Dense ,Dropout, Flatten, Activation, MaxPooling3D, Input


In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    print("reading from cloud...")
    path = "/content/drive/MyDrive/Colab Notebooks/MPPE/data_70p_overhang/stls_opt/"
    path_vox = "/content/drive/MyDrive/Colab Notebooks/MPPE/data_70p_overhang/voxels/"
    
except:
    import pathlib
    print("reading from disk")
    path = str(pathlib.Path().resolve())+"/dataset/stls_opt/"
    path_vox = str(pathlib.Path().resolve())+"/dataset/voxels/"

Mounted at /content/drive
reading from cloud...


In [ ]:
mesh = trimesh.load(path+"63_8475_10313_17790_28845.stl")

In [28]:
name = "63_8475_10313_17790_28845.stl"

In [29]:
data = name.replace(".","_").split("_")
data = int(data[:-1])

TypeError: ignored

In [48]:
def calculate_cost(filename):
  #params
  #time - parts usally need 15 minutes for preparation
  #1h - 10eur
  price_per_h = 2
  price_per_meter = 0.17#0.1133 #150 meters costs 17e 
  filament_diameter = 2.85 #mm
  filament_phi = np.power(filament_diameter/2,2)*np.pi #mm^2
  
  support_removal_multiplier= 2
  support_removal_multiplier_time = 1.5
  profit_margin = 0.1
  #larger parts need longer time + 5 minutes slice time. 
  
  #get data from filename
  data = filename.replace(".","_").split("_")
  data = [int(x) for x in data[:-1]]

  #calculate cost from filament lenght

  #calulate cost no support 
  cost_filament = ((data[2]/(filament_phi*1000)))*price_per_meter

  #calulcate cost of printing supports only 
  cost_supports = (abs(data[2]-data[4])/(filament_phi*1000))*price_per_meter
  cost_supports_removal = support_removal_multiplier*cost_supports

  #calulate timewise cost in hours
  print_time = data[1]
  print_time_supports = data[3]

  #convert to horus and include time for removing supports 
  print_time= (((print_time + 2*abs(print_time-print_time_supports))*support_removal_multiplier_time)/3600)
  print_time_cost = print_time * price_per_h

  #cena
  cost = (cost_filament+cost_supports+cost_supports_removal+print_time_cost)
  cost = (cost*profit_margin)+cost

  return cost

#cost = calculate_cost("89_88123_134991_106833_152824.stl")
#cost = calculate_cost("62_1605_2129_2398_3618.stl")
cost = calculate_cost("108_31956_48635_34435_50936.stl")
print(cost)

35.46582523917991


In [ ]:
#a = np.zeros(0)
a = np.append(a,mesh.volume+mesh.volume*2)
print(a)

In [ ]:
X_train = np.zeros([0,100,100,100])
X_train_volume = np.zeros([0,1])
X_train_area = np.zeros([0,1])

y_train = np.zeros(0)
for filename in os.listdir(path_vox):
    if filename.endswith(".npy"):
        #print("appending", filename.split(".")[0])
        vox = np.load(path_vox+filename)
        X_train = np.append(X_train,vox[np.newaxis,...],axis=0)
        y_train = np.append(y_train,calculate_cost(filename))
        
        mesh = trimesh.load(path+filename.split(".")[0]+".stl")
        
        mv = np.array(abs(mesh.volume))
        X_train_volume = np.append(X_train_volume,mv[np.newaxis,...])

        ma = np.array(abs(mesh.area))
        X_train_area = np.append(X_train_area,ma[np.newaxis,...])

X_train_area = X_train_area[...,np.newaxis]
X_train_volume = X_train_volume[...,np.newaxis]
X_train = X_train[...,np.newaxis]

       

KeyboardInterrupt: ignored

In [ ]:
#my net
input1 = keras.Input(shape=(X_train.shape[1:]))
input2 = keras.Input(shape=(X_train_volume.shape[1:]))

y = Dense(1)(input2)

x = Conv3D(32,(3,3,3),strides = (2,2,2), activation='relu', padding = 'same')(input1)
x = Conv3D(32,(3,3,3),strides = (2,2,2), activation='relu', padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = Conv3D(16,(3,3,3),strides = (2,2,2), activation='relu', padding = 'same')(x)
x = Conv3D(16,(3,3,3),strides = (2,2,2), activation='relu', padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = Conv3D(8,(3,3,3),strides = (2,2,2), activation='relu', padding = 'same')(x)
x = Conv3D(8,(3,3,3),strides = (2,2,2), activation='relu', padding = 'same')(x)

x = MaxPooling3D(pool_size=(1, 1, 1))(x)

x = Activation('relu')(x)

x = layers.Dropout(0.8)(x)

x = Flatten()(x)

x = layers.Concatenate(axis=1)([x,input2])

dense = Dense(2000, activation = 'relu')(x)
dense = Dense(300, activation = 'relu')(dense)
dense = Dense(150, activation = 'relu')(dense)
dense = Dense(20, activation = 'relu')(dense)
dense = Dense(16, activation = 'relu')(dense)
    # final layer with 10 neurons to classify the instances
output = Dense(1, activation = 'linear')(dense)
    
#outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=[input1,input2], outputs=output, name="pointnet")

In [ ]:
y_train = np.float64(y_train)
y_train_max = y_train.max()

X_train_volume_max = X_train_volume.max()

model.compile(
    loss="MSE",
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["MSE"],
)

model.fit([X_train, np.log(X_train_volume)], np.log(y_train), batch_size = 8, epochs = 100, verbose = 1,validation_split=0.2)


Epoch 1/100
19/19 [==============================] - 13s 173ms/step - loss: 90.8961 - MSE: 90.8961 - val_loss: 86.2751 - val_MSE: 86.2751
Epoch 2/100
19/19 [==============================] - 2s 114ms/step - loss: 87.8257 - MSE: 87.8257 - val_loss: 83.4837 - val_MSE: 83.4837
Epoch 3/100
19/19 [==============================] - 2s 114ms/step - loss: 85.3339 - MSE: 85.3339 - val_loss: 81.5071 - val_MSE: 81.5071
Epoch 4/100
19/19 [==============================] - 2s 114ms/step - loss: 83.4044 - MSE: 83.4044 - val_loss: 79.6319 - val_MSE: 79.6319
Epoch 5/100
19/19 [==============================] - 2s 113ms/step - loss: 81.5276 - MSE: 81.5276 - val_loss: 77.8506 - val_MSE: 77.8506
Epoch 6/100
19/19 [==============================] - 2s 114ms/step - loss: 79.6893 - MSE: 79.6893 - val_loss: 76.0425 - val_MSE: 76.0425
Epoch 7/100
19/19 [==============================] - 2s 113ms/step - loss: 77.8270 - MSE: 77.8270 - val_loss: 74.1460 - val_MSE: 74.1460
Epoch 8/100
19/19 [=====================

In [ ]:
y_train_max = y_train.max()
preds = model.predict([X_train, np.log(X_train_volume)])#*y_train_max

In [ ]:
np.sum(((preds)-(np.log(y_train))))/len(preds)

NameError: ignored

In [ ]:
np.exp(preds)

array([[7.87043115e+03],
       [2.47031973e+04],
       [4.98157148e+04],
       [5.74234229e+03],
       [5.46663037e+03],
       [3.95006152e+03],
       [2.39162754e+04],
       [6.92928271e+03],
       [8.90610625e+04],
       [3.27020361e+03],
       [3.08263403e+03],
       [6.14138135e+03],
       [1.88391582e+04],
       [1.19131807e+04],
       [2.51163730e+04],
       [1.11226318e+04],
       [1.71669531e+03],
       [7.69594453e+04],
       [6.46361963e+03],
       [5.49705664e+03],
       [3.43621758e+04],
       [2.28198086e+04],
       [1.17278641e+02],
       [1.36447168e+04],
       [5.33147754e+03],
       [5.38385449e+03],
       [2.91393896e+03],
       [7.73897314e+03],
       [7.38452031e+04],
       [4.13495195e+04],
       [1.29137158e+04],
       [5.52675049e+03],
       [6.70803857e+03],
       [2.09610596e+03],
       [1.16563281e+04],
       [9.48935156e+03],
       [1.07746885e+04],
       [7.35390332e+03],
       [3.87554727e+04],
       [1.22870088e+04],


In [ ]:
y_train

array([1.77900e+04, 5.95770e+04, 4.92700e+04, 8.83500e+03, 6.03500e+03,
       4.37000e+03, 3.31020e+04, 5.99600e+03, 9.73090e+04, 2.56400e+03,
       4.73700e+03, 6.96500e+03, 3.10130e+04, 1.37360e+04, 1.15430e+04,
       7.54400e+03, 1.36500e+03, 6.75540e+04, 9.44700e+03, 3.54000e+03,
       4.89190e+04, 1.95850e+04, 1.71000e+02, 7.86300e+03, 3.83800e+03,
       6.57200e+03, 5.51800e+03, 9.58800e+03, 6.42140e+04, 5.96900e+04,
       1.31260e+04, 4.38500e+03, 8.71600e+03, 1.99300e+03, 1.15240e+04,
       1.11980e+04, 1.41810e+04, 5.38200e+03, 4.07120e+04, 1.28960e+04,
       9.47500e+03, 3.51300e+03, 5.78600e+04, 1.48020e+04, 1.43660e+04,
       3.24590e+04, 1.22200e+03, 3.58310e+04, 6.45000e+02, 2.70326e+05,
       4.19700e+03, 1.14450e+04, 1.69310e+04, 6.18070e+04, 9.44000e+02,
       9.26500e+04, 8.70320e+04, 6.00700e+03, 1.37260e+04, 3.50000e+02,
       4.42070e+04, 1.39320e+04, 1.05800e+03, 7.27800e+03, 1.11230e+04,
       1.30809e+05, 6.98400e+03, 9.36000e+02, 8.43100e+03, 1.096